In [57]:
import os
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
def unfuck(raw_html):
    raw_html = str(raw_html)
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext.replace('\t', '').replace('\n', '').strip(' ')

In [77]:
out = []

In [78]:
for file in os.listdir('./scratch/'):
    soup = BeautifulSoup(open('./scratch/' + file, encoding='utf-8'), 'html.parser')
    
    ks = ['tmp', 'REDIZO', 'nazev', 'typ']
    keys = list(map(unfuck, soup.find('thead').find_all('th')[2:]))
    ks.extend(keys)
    for row in soup.find_all('tr'):
        x = row.find('td', text=re.compile('REDIZO'))
        if x != None:
            tmp = []
            rw = x.parent.find_all('td')
            for r in rw:
                tmp.append(r.text)
            out.append(dict(zip(ks, tmp)))

In [81]:
dta = pd.DataFrame.from_dict(out)

In [84]:
def nabodenicka_pdp(val):
    return val.replace(',', '.')

In [86]:
dta = dta.applymap(nabodenicka_pdp)

In [87]:
ne = ['nazev', 'tmp', 'typ']
for col in dta.columns:
    if col not in  ne:
        dta[col] = pd.to_numeric(dta[col], errors = 'coerce')

In [92]:
dta.drop_duplicates(inplace=True)

In [93]:
dta.to_csv('./data/data_dump.csv', encoding='utf-8', index=False)